<a href="https://colab.research.google.com/github/ishikaarora/pytorch-tutorials/blob/master/Defense_final_Exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function, division
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
import random
import pandas as pd
import numpy as np
from scipy import interp
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
plt.ion()   # interactive mode
from google.colab import drive
from torchvision.datasets import CIFAR10


##  Model classes

In [0]:
class mymodel(nn.Module):
    """Custom Model"""

    def __init__(self, in_features, hidden_dim, n_classes):
        """Custom Model Builder."""
        super(mymodel, self).__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim//2)
        self.fc3 = nn.Linear(hidden_dim//2, n_classes)
   
  
    def forward(self, images):
        """Perform forward."""
        
        images_vec = images.view(images.size(0), -1)
        hidden_out = F.relu(self.fc1(images_vec))
        hidden_out = F.relu(self.fc2(hidden_out))
        scores = self.fc3(hidden_out)

        return scores

## Constants

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root_path = 'gdrive/My Drive/Colab Notebooks/Adversarial/'

In [7]:
#mounting drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def transfer_attack(proxy_model, data, target, **kwargs):
    # Attack setup
    loss_fn = kwargs.get('loss_fn', nn.CrossEntropyLoss())
    epsilon = kwargs.get('epsilon', 0.1)
    data.requires_grad = True
    
    proxy_output = proxy_model(data)
    # Calculate the loss, forward pass
    loss = loss_fn(proxy_output, target)
    # Zero all existing gradients
    proxy_model.zero_grad()
    # Calculate gradients of model in backward pass
    loss.backward()
    # Collect datagrad
    data_grad = data.grad.data
    
    # Call FGSM Attack
    perturbed_data = gradient_attack(data, epsilon, data_grad)
    
    return perturbed_data

## Atack functions

In [0]:
# FGSM Gradient attack code
def gradient_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    # perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

### FGSM Attack

In [0]:
# FGSM Attack
def fgsm_attack(model, data, target, **kwargs):
    loss_fn = kwargs.get('loss_fn', nn.CrossEntropyLoss())
    epsilon = kwargs.get('epsilon', 0.1)
    data.requires_grad = True
    
    output = model(data)
    loss = loss_fn(output,target) 
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    perturbed_data = gradient_attack(data, epsilon, data_grad)

    return perturbed_data

### CW Attack

In [0]:
def cw_attack(model, images, labels, **kwargs):
  cw_attack_model = torchattacks.CW(model)
  adversarial_images = cw_attack_model(images, labels)
  return adversarial_images

### Transferability Attack

Load pretrained model on CIFAR-10

In [0]:
# Load the proxy model for Transfer attack
proxy_model = torch.load(root_path + "models/myproxymodel_resnet.pt", map_location=torch.device(device))
# proxy_model = torch.load("myproxymodel_resnet.pt", map_location=torch.device(device))
# We don't want to train the model, so tell PyTorch not to compute gradients
# with respect to model parameters.
for param in proxy_model.parameters():
    param.requires_grad = False

In [0]:
proxy_acc, data_size = calculate_accuracy(proxy_model, testloader)
print(proxy_acc, data_size)

Current Accuracy: 792 / 1000 = 79.2
Current Accuracy: 1585 / 2000 = 79.25
Current Accuracy: 2380 / 3000 = 79.33333333333333
Current Accuracy: 3158 / 4000 = 78.95
Current Accuracy: 3970 / 5000 = 79.4
Current Accuracy: 4767 / 6000 = 79.45
Current Accuracy: 5541 / 7000 = 79.15714285714286
Current Accuracy: 6355 / 8000 = 79.4375
Current Accuracy: 7140 / 9000 = 79.33333333333333
Current Accuracy: 7946 / 10000 = 79.46
79.46 10000


## Util functions 

In [0]:
def class_report(y_true, y_pred, y_score=None, average='micro'):
    print(type(y_pred),type(y_score),y_pred.shape,y_score.shape)
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum()
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred'] = pred_cnt
    class_report_df['pred'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int),
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(),
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(),
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"],
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

In [0]:
# Only for FGSM and transfer attacks
def create_adversarial(model, images, labels, attack_type=None, **kwargs):
    if attack_type is not None:
        if attack_type == "fgsm":
            perturbed_images = fgsm_attack(model, images, labels, **kwargs)
        elif attack_type == "transfer":
            perturbed_images = transfer_attack(proxy_model, images, labels, **kwargs)
        else:
            perturbed_images = cw_attack(model, images, labels, **kwargs)
    else:    
      idx = random.uniform(0,1)

      if idx < 0.5:
          perturbed_images = fgsm_attack(model, images, labels, **kwargs)
      else:
          perturbed_images = transfer_attack(proxy_model, images, labels, **kwargs)

    return perturbed_images

In [0]:
def attack_accuracy(model, loader, attack_type=None, print_freq=1000, epsilon=0.1, run_attack=False):
    model.eval()
    total = 0
    correct = 0
    show_im = True
    all_labels = []
    all_preds = []
    all_probs = []
    #batch_count = 0 

    for data in loader:
        #batch_count+=1
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        
        if run_attack:
          kwargs = {'loss_fn': nn.CrossEntropyLoss(), 'epsilon': epsilon}     
          images = create_adversarial(model, images, labels, attack_type=attack_type, **kwargs)

        with torch.no_grad():
            outputs = model(images)
            out_prob = F.softmax(outputs, dim=1)

        _,predicted = torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()

        all_labels.append(labels.cpu().numpy())
        all_preds.append(predicted.cpu().numpy())
        all_probs.append(out_prob.cpu().numpy())

        if total % print_freq == 0:
            print("Current Accuracy: {} / {} = {}".format(correct, total, correct/total*100))
        #if batch_count>=test_num_batches:
        #    break

    
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    all_probs = np.concatenate(all_probs)
    print(all_labels.shape,all_preds.shape,all_probs.shape)
    report_with_auc = class_report(
                            y_true = all_labels,
                            y_pred = all_preds,
                            y_score= all_probs)
           
    return (correct/total)*100, total, report_with_auc

## Load pre trained model

In [0]:
model = torch.load(root_path+"models/mymodel.pt", map_location=torch.device(device))
for param in model.parameters():
    param.requires_grad = False

## Load pre trained defense model

In [0]:
distil_model = torch.load(root_path+"models/my_distil_model1.pt", map_location=torch.device(device))
for param in distil_model.parameters():
    param.requires_grad = False

## Preparing test data sets

### Load labels

In [0]:
labels = torch.load(root_path+"samples/all_labels.pt", map_location=torch.device(device))

In [0]:
labels_1200 = labels[0:1200]
labels_1200_3099 = labels[1200:3100]
labels_3100_4999= labels[3100:5000]

### Load FGSM

In [0]:
fgsm_images = torch.load(root_path+"samples/fgsm_images_010_1200-3099.pt", map_location=torch.device(device))
fgsm_dataset = TensorDataset(fgsm_images, labels_1200_3099)
fgsm_loader = DataLoader(fgsm_dataset, batch_size= 4, shuffle=False)

In [0]:
fgsm_images1 = torch.load(root_path+"samples/fgsm_images_015_1200-3099.pt", map_location=torch.device(device))
fgsm_dataset1 = TensorDataset(fgsm_images1, labels_1200_3099)
fgsm_loader1 = DataLoader(fgsm_dataset1, batch_size= 4, shuffle=False)

### Load Transfer Attack Samples

In [0]:
transfer_images = torch.load(root_path+"samples/transfer_images_010_3100-4999.pt", map_location=torch.device(device))
transfer_dataset = TensorDataset(transfer_images, labels_3100_4999)
transfer_loader = DataLoader(transfer_dataset, batch_size= 4, shuffle=False)

In [0]:
transfer_images1 = torch.load(root_path+"samples/transfer_images_015_3100-4999.pt", map_location=torch.device(device))
transfer_dataset1 = TensorDataset(transfer_images1, labels_3100_4999)
transfer_loader1 = DataLoader(transfer_dataset1, batch_size= 4, shuffle=False)

### Load CW data

In [0]:
cw_images = torch.load(root_path+"samples/cw_images.pt", map_location=torch.device(device))
cw_labels = torch.load(root_path+"samples/cw_labels.pt", map_location=torch.device(device))
cw_dataset = TensorDataset(cw_images, cw_labels)
cw_loader = DataLoader(cw_dataset, batch_size= 4, shuffle=False)

### Load Mixture Data set

Mixed with 1st 1200 CW, next 1900 FGSM, next 1900 Transfer (eps=0.10)


In [0]:
mixed_images = torch.cat([cw_images,fgsm_images,transfer_images],dim=0)
# mixed_labels = torch.cat([labels_1200,labels_3100_4999,cw_labels],dim=0)
mixed_labels = labels[0:5000]
mixed_dataset = TensorDataset(mixed_images, mixed_labels)
mixed_loader = DataLoader(mixed_dataset, batch_size= 4, shuffle=False)

In [0]:
mixed_images1 = torch.cat([cw_images,fgsm_images1,transfer_images1],dim=0)
# mixed_labels = torch.cat([labels_1200,labels_3100_4999,cw_labels],dim=0)
mixed_labels = labels[0:5000]
mixed_dataset1 = TensorDataset(mixed_images1, mixed_labels)
mixed_loader1 = DataLoader(mixed_dataset1, batch_size= 4, shuffle=False)

### Load normal samples

In [18]:
cifar10_mean = [0.49139968 , 0.48215841 , 0.44653091]
cifar10_std = [0.24703223,  0.24348513 , 0.26158784]

#Define transformations for the test set
transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean,cifar10_std)
])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform=transform_test)

testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

0it [00:00, ?it/s]

170500096it [00:03, 45243884.98it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data


## Test : Distilled model + Normal Samples + No Detector

In [20]:
normal_sample_acc, normal_sample_data_size,normal_sample_report_with_auc = attack_accuracy(distil_model, testloader, print_freq=1000)

Current Accuracy: 920 / 1000 = 92.0
Current Accuracy: 1828 / 2000 = 91.4
Current Accuracy: 2735 / 3000 = 91.16666666666666
Current Accuracy: 3623 / 4000 = 90.575
Current Accuracy: 4532 / 5000 = 90.64
Current Accuracy: 5443 / 6000 = 90.71666666666667
Current Accuracy: 6354 / 7000 = 90.77142857142857
Current Accuracy: 7271 / 8000 = 90.8875
Current Accuracy: 8183 / 9000 = 90.92222222222223
Current Accuracy: 9094 / 10000 = 90.94
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [21]:
normal_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.921627,0.9290,0.925299,1000.0,1008.0,0.991657
1,0.978471,0.9090,0.942457,1000.0,929.0,0.992619
2,0.876673,0.9170,0.896383,1000.0,1046.0,0.992878
3,0.858193,0.8170,0.837090,1000.0,952.0,0.977505
4,0.960875,0.8350,0.893526,1000.0,869.0,0.973687
5,0.866799,0.8720,0.869392,1000.0,1006.0,0.986979
6,0.953893,0.9310,0.942308,1000.0,976.0,0.994342
7,0.866063,0.9570,0.909264,1000.0,1105.0,0.992774
8,0.911488,0.9680,0.938894,1000.0,1062.0,0.995027
9,0.915950,0.9590,0.936981,1000.0,1047.0,0.995349


## Test: Distilled model + CW attack samples + No detector

In [0]:
cw_sample_acc, cw_sample_data_size,cw_sample_report_with_auc = attack_accuracy(distil_model, cw_loader, print_freq=100)

Current Accuracy: 44 / 100 = 44.0
Current Accuracy: 84 / 200 = 42.0
Current Accuracy: 113 / 300 = 37.666666666666664
Current Accuracy: 150 / 400 = 37.5
Current Accuracy: 194 / 500 = 38.800000000000004
Current Accuracy: 232 / 600 = 38.666666666666664
Current Accuracy: 271 / 700 = 38.714285714285715
Current Accuracy: 311 / 800 = 38.875
Current Accuracy: 344 / 900 = 38.22222222222222
Current Accuracy: 379 / 1000 = 37.9
Current Accuracy: 422 / 1100 = 38.36363636363636
Current Accuracy: 457 / 1200 = 38.083333333333336
(1200,) (1200,) (1200, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (1200,) (1200, 10)


In [0]:
cw_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.263456,0.720930,0.385892,129.0,353.0,0.842037
1,1.000000,0.026316,0.051282,114.0,3.0,0.652628
2,0.294872,0.584746,0.392045,118.0,234.0,0.794221
3,0.267490,0.511811,0.351351,127.0,243.0,0.749345
4,0.781250,0.240385,0.367647,104.0,32.0,0.760633
5,0.434211,0.330000,0.375000,100.0,76.0,0.727009
6,0.612069,0.503546,0.552529,141.0,116.0,0.845699
7,0.571429,0.512821,0.540541,117.0,105.0,0.853122
8,1.000000,0.277778,0.434783,126.0,35.0,0.796710
9,1.000000,0.024194,0.047244,124.0,3.0,0.619589


## Attacks (Epsilon=0.10)

### Test : Distilled model + FGSM + No Detector!

In [23]:
fgsm_acc, data_size, fgsm_report_with_auc = attack_accuracy(distil_model, testloader, attack_type="fgsm", print_freq=1000, epsilon=0.10, run_attack=True)

Current Accuracy: 392 / 1000 = 39.2
Current Accuracy: 790 / 2000 = 39.5
Current Accuracy: 1172 / 3000 = 39.06666666666666
Current Accuracy: 1580 / 4000 = 39.5
Current Accuracy: 1971 / 5000 = 39.42
Current Accuracy: 2321 / 6000 = 38.68333333333333
Current Accuracy: 2697 / 7000 = 38.52857142857143
Current Accuracy: 3094 / 8000 = 38.675
Current Accuracy: 3441 / 9000 = 38.233333333333334
Current Accuracy: 3844 / 10000 = 38.440000000000005
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [24]:
fgsm_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.925651,0.2490,0.392435,1000.0,269.0,0.868408
1,0.981043,0.2070,0.341866,1000.0,211.0,0.789249
2,0.168559,0.8610,0.281925,1000.0,5108.0,0.823054
3,0.387911,0.5070,0.439532,1000.0,1307.0,0.845103
4,0.620168,0.3690,0.462696,1000.0,595.0,0.868579
5,0.900000,0.0900,0.163636,1000.0,100.0,0.710487
6,0.486998,0.6180,0.544733,1000.0,1269.0,0.898234
7,0.745020,0.3740,0.498003,1000.0,502.0,0.818227
8,0.893064,0.3090,0.459138,1000.0,346.0,0.899081
9,0.887372,0.2600,0.402166,1000.0,293.0,0.858204


### Test: Distilled model + Transfer Attack Samples + No Detector

In [25]:
transfer_sample_acc, transfer_sample_data_size,transfer_sample_report_with_auc = attack_accuracy(distil_model, testloader, attack_type="transfer", 
                                                                                                 print_freq=1000, epsilon=0.10, run_attack=True)

Current Accuracy: 421 / 1000 = 42.1
Current Accuracy: 847 / 2000 = 42.35
Current Accuracy: 1274 / 3000 = 42.46666666666667
Current Accuracy: 1721 / 4000 = 43.025000000000006
Current Accuracy: 2150 / 5000 = 43.0
Current Accuracy: 2528 / 6000 = 42.13333333333333
Current Accuracy: 2935 / 7000 = 41.92857142857142
Current Accuracy: 3364 / 8000 = 42.05
Current Accuracy: 3759 / 9000 = 41.766666666666666
Current Accuracy: 4188 / 10000 = 41.88
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [26]:
transfer_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.937870,0.3170,0.473842,1000.0,338.0,0.888563
1,0.984848,0.2600,0.411392,1000.0,264.0,0.835453
2,0.176752,0.9230,0.296689,1000.0,5222.0,0.867877
3,0.395315,0.5400,0.456467,1000.0,1366.0,0.869287
4,0.704846,0.3200,0.440165,1000.0,454.0,0.874523
5,0.853107,0.1510,0.256585,1000.0,177.0,0.759091
6,0.637048,0.4230,0.508413,1000.0,664.0,0.890551
7,0.765101,0.4560,0.571429,1000.0,596.0,0.861624
8,0.886994,0.4160,0.566372,1000.0,469.0,0.917841
9,0.848889,0.3820,0.526897,1000.0,450.0,0.903220


### Test: Normal Model + FGSM+Transfer Attack + No detector

In [27]:
fgsm_transfer_sample_acc, fgsm_transfer_sample_data_size,fgsm_transfer_sample_report_with_auc = attack_accuracy(distil_model, testloader, 
                                                                                                 print_freq=1000, epsilon=0.10, run_attack=True)

Current Accuracy: 417 / 1000 = 41.699999999999996
Current Accuracy: 824 / 2000 = 41.199999999999996
Current Accuracy: 1229 / 3000 = 40.96666666666667
Current Accuracy: 1666 / 4000 = 41.65
Current Accuracy: 2067 / 5000 = 41.339999999999996
Current Accuracy: 2431 / 6000 = 40.516666666666666
Current Accuracy: 2825 / 7000 = 40.35714285714286
Current Accuracy: 3239 / 8000 = 40.4875
Current Accuracy: 3606 / 9000 = 40.06666666666667
Current Accuracy: 4015 / 10000 = 40.150000000000006
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [28]:
fgsm_transfer_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.942373,0.2780,0.429344,1000.0,295.0,0.880843
1,0.978903,0.2320,0.375101,1000.0,237.0,0.811999
2,0.172112,0.8850,0.288180,1000.0,5142.0,0.844486
3,0.398230,0.5400,0.458404,1000.0,1356.0,0.859433
4,0.655617,0.3560,0.461439,1000.0,543.0,0.868810
5,0.851064,0.1200,0.210342,1000.0,141.0,0.729898
6,0.522177,0.5180,0.520080,1000.0,992.0,0.888654
7,0.782197,0.4130,0.540576,1000.0,528.0,0.840048
8,0.890274,0.3570,0.509636,1000.0,401.0,0.908233
9,0.865753,0.3160,0.463004,1000.0,365.0,0.878165


### Test: Distilled model + Mixed attack samples + No detector

In [0]:
mixed_sample_acc, mixed_sample_data_size,mixed_sample_report_with_auc = attack_accuracy(distil_model, mixed_loader, print_freq=1000)

Current Accuracy: 379 / 1000 = 37.9
Current Accuracy: 531 / 2000 = 26.55
Current Accuracy: 653 / 3000 = 21.766666666666666
Current Accuracy: 751 / 4000 = 18.775
Current Accuracy: 846 / 5000 = 16.919999999999998
(5000,) (5000,) (5000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (5000,) (5000, 10)


In [0]:
mixed_sample_report_with_auc

,precision,recall,f1-score,support,pred,AUC
0,0.216274,0.206967,0.211518,488.0,467.0,0.577710
1,0.147059,0.029703,0.049423,505.0,102.0,0.539028
2,0.126200,0.539062,0.204520,512.0,2187.0,0.584774
3,0.153034,0.233400,0.184861,497.0,758.0,0.550853
4,0.189394,0.098619,0.129702,507.0,264.0,0.575073
5,0.277372,0.077869,0.121600,488.0,137.0,0.549126
6,0.221505,0.209776,0.215481,491.0,465.0,0.594293
7,0.265781,0.161616,0.201005,495.0,301.0,0.585263
8,0.293478,0.107143,0.156977,504.0,184.0,0.565765
9,0.096296,0.025341,0.040123,513.0,135.0,0.519295


## Attacks (Epsilon=0.15)

### Test : Distilled model + FGSM + No Detector! (eps=0.15)

In [0]:
fgsm_acc1, data_size, fgsm_report_with_auc1 = attack_accuracy(distil_model, testloader, attack_type="fgsm", print_freq=1000, epsilon=0.15, run_attack=True)

Current Accuracy: 250 / 1000 = 25.0
Current Accuracy: 474 / 2000 = 23.7
Current Accuracy: 699 / 3000 = 23.3
Current Accuracy: 932 / 4000 = 23.3
Current Accuracy: 1145 / 5000 = 22.900000000000002
Current Accuracy: 1351 / 6000 = 22.516666666666666
Current Accuracy: 1580 / 7000 = 22.57142857142857
Current Accuracy: 1801 / 8000 = 22.5125
Current Accuracy: 2013 / 9000 = 22.366666666666667
Current Accuracy: 2244 / 10000 = 22.439999999999998
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [0]:
fgsm_report_with_auc1

,precision,recall,f1-score,support,pred,AUC
0,0.892308,0.0580,0.108920,1000.0,65.0,0.808396
1,1.000000,0.0440,0.084291,1000.0,44.0,0.694241
2,0.120614,0.8490,0.211220,1000.0,7039.0,0.716015
3,0.350402,0.3490,0.349699,1000.0,996.0,0.788136
4,0.567376,0.2400,0.337316,1000.0,423.0,0.841091
5,0.888889,0.0080,0.015857,1000.0,9.0,0.582294
6,0.415039,0.4250,0.419960,1000.0,1024.0,0.837048
7,0.582734,0.1620,0.253521,1000.0,278.0,0.719690
8,0.960000,0.0480,0.091429,1000.0,50.0,0.849705
9,0.847222,0.0610,0.113806,1000.0,72.0,0.830946


### Test: Distilled model + Transfer Attack Samples + No Detector (eps=0.15)

In [0]:
transfer_sample_acc1, transfer_sample_data_size,transfer_sample_report_with_auc1 = attack_accuracy(distil_model, testloader, attack_type="transfer", 
                                                                                                 print_freq=1000, epsilon=0.15, run_attack=True)

Current Accuracy: 238 / 1000 = 23.799999999999997
Current Accuracy: 474 / 2000 = 23.7
Current Accuracy: 714 / 3000 = 23.799999999999997
Current Accuracy: 953 / 4000 = 23.825
Current Accuracy: 1180 / 5000 = 23.599999999999998
Current Accuracy: 1385 / 6000 = 23.083333333333332
Current Accuracy: 1614 / 7000 = 23.057142857142857
Current Accuracy: 1839 / 8000 = 22.9875
Current Accuracy: 2043 / 9000 = 22.7
Current Accuracy: 2281 / 10000 = 22.81
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [0]:
transfer_sample_report_with_auc1

,precision,recall,f1-score,support,pred,AUC
0,0.888889,0.0800,0.146789,1000.0,90.0,0.813791
1,0.983607,0.0600,0.113101,1000.0,61.0,0.721687
2,0.122743,0.9380,0.217079,1000.0,7642.0,0.768628
3,0.378981,0.3570,0.367662,1000.0,942.0,0.792618
4,0.677551,0.1660,0.266667,1000.0,245.0,0.841528
5,0.916667,0.0220,0.042969,1000.0,24.0,0.622966
6,0.560372,0.1810,0.273621,1000.0,323.0,0.805134
7,0.596059,0.2420,0.344239,1000.0,406.0,0.773419
8,0.915888,0.0980,0.177055,1000.0,107.0,0.861216
9,0.856250,0.1370,0.236207,1000.0,160.0,0.858696


### Test: Distilled model + FGSM+Transfer Attack Samples + No Detector (eps=0.15)

In [0]:
fgsm_transfer_sample_acc1, fgsm_transfer_sample_data_size1,fgsm_transfer_sample_report_with_auc1 = attack_accuracy(distil_model, testloader, 
                                                                                                 print_freq=1000, epsilon=0.15, run_attack=True)

Current Accuracy: 248 / 1000 = 24.8
Current Accuracy: 487 / 2000 = 24.349999999999998
Current Accuracy: 726 / 3000 = 24.2
Current Accuracy: 957 / 4000 = 23.925
Current Accuracy: 1183 / 5000 = 23.66
Current Accuracy: 1385 / 6000 = 23.083333333333332
Current Accuracy: 1615 / 7000 = 23.07142857142857
Current Accuracy: 1832 / 8000 = 22.900000000000002
Current Accuracy: 2042 / 9000 = 22.68888888888889
Current Accuracy: 2276 / 10000 = 22.759999999999998
(10000,) (10000,) (10000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (10000,) (10000, 10)


In [0]:
fgsm_transfer_sample_report_with_auc1

,precision,recall,f1-score,support,pred,AUC
0,0.851852,0.0690,0.127660,1000.0,81.0,0.809337
1,1.000000,0.0510,0.097050,1000.0,51.0,0.702074
2,0.121187,0.8860,0.213211,1000.0,7311.0,0.741545
3,0.369634,0.3530,0.361125,1000.0,955.0,0.795568
4,0.605797,0.2090,0.310781,1000.0,345.0,0.836655
5,0.952381,0.0200,0.039177,1000.0,21.0,0.601015
6,0.448773,0.3110,0.367395,1000.0,693.0,0.805458
7,0.589595,0.2040,0.303120,1000.0,346.0,0.751740
8,0.913580,0.0740,0.136910,1000.0,81.0,0.857030
9,0.853448,0.0990,0.177419,1000.0,116.0,0.840678


### Test: Distilled model + Mixed attack samples + No detector (eps=0.15)

In [0]:
mixed_sample_acc1, mixed_sample_data_size,mixed_sample_report_with_auc1 = attack_accuracy(distil_model, mixed_loader1, print_freq=1000)

Current Accuracy: 379 / 1000 = 37.9
Current Accuracy: 543 / 2000 = 27.150000000000002
Current Accuracy: 660 / 3000 = 22.0
Current Accuracy: 757 / 4000 = 18.925
Current Accuracy: 853 / 5000 = 17.06
(5000,) (5000,) (5000, 10)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> (5000,) (5000, 10)


In [0]:
mixed_sample_report_with_auc1

,precision,recall,f1-score,support,pred,AUC
0,0.251309,0.196721,0.220690,488.0,382.0,0.573498
1,0.166667,0.007921,0.015123,505.0,24.0,0.535813
2,0.120013,0.710938,0.205360,512.0,3033.0,0.579762
3,0.168810,0.211268,0.187668,497.0,622.0,0.563221
4,0.259259,0.082840,0.125561,507.0,162.0,0.579509
5,0.379310,0.067623,0.114783,488.0,87.0,0.555177
6,0.257790,0.185336,0.215640,491.0,353.0,0.597189
7,0.334783,0.155556,0.212414,495.0,230.0,0.612598
8,0.606557,0.073413,0.130973,504.0,61.0,0.592949
9,0.086957,0.007797,0.014311,513.0,46.0,0.520884
